## Import dependencies

In [ ]:
import pandas as pd
from IPython.display import HTML
import requests
import json
from scipy.spatial import distance

from helper_function.notebook_helpers import show_vehicle_routes, get_minutes_from_datetime
from helper_function.map_helpers import get_map_by_vehicle

from cuopt_thin_client import CuOptServiceClient

## Read input data from CSV files

Suppose you are working as an Optimization Scientist at a grocery chain throughout New York City. There are 97 stores and 3 distribution centers. Every day, stores place an order for food that must be delivered the next day to ensure they are fully stocked. Given input data about stores' orders, distribution centers, and the available fleet of vehicles, it is your job to calculate the route for each vehicle such that all orders are fulfilled while minimizing vehicles' travel time and cost. For a problem space of 100 locations there are about 100! possible routes. You can do the math yourself- but that's a lot. Lucky for you, you have access to the cuOpt solver. All you need to do is read the input data and preprocess the data. Once all the data is ready, you just need to save it to one Python dictionary and send it to cuOpt, which does all the hard computation for you.

Let's walk through these steps. 

For the Last Mile Delivery (LMD) use case, we need 3 datasets with the following features:

- Depots
    - Name
    - Location
    - Start and end time (operation hours)
- Orders
    - Store Name
    - Location
    - Start and end time (store hours)
    - Demand
    - Service time
    - Loyalty Member
    - Delivery Requirement
- Vehicles
    - Name/ID Number
    - Assigned depot
    - Start and end time (vehicle/driver shift hours)
    - Break time
    - Capacity
    - Max time
    
You may have additional features depending on the problem at hand.

Location data needs to be in the form of coordinate points (longitude and latitude values). Our data already has coordinate points included. If you're using new data and need to do so yourself, you will need to use a third party tool.


In this workflow, we are using locations from the following Kaggle dataset https://www.kaggle.com/datasets/arianazmoudeh/airbnbopendata. This is a dataset of Airbnbs in New York City. Our problem space has 100 locations total which includes 3 depots and 97 orders. The coordinate points are taken from the dataset and the rest of the features are synthetic data. We have 15 vehicles available.

In [ ]:
DATA_PATH = "data/"

orders_df = pd.read_csv(DATA_PATH+"orders_lmd.csv")
depots_df = pd.read_csv(DATA_PATH+"depots_lmd.csv")
vehicles_df = pd.read_csv(DATA_PATH+"vehicles_lmd.csv")

In [ ]:
n_depots = len(depots_df.index)
n_orders = len(orders_df.index)
n_vehicles = len(vehicles_df.index)

n_loc_total = n_orders + n_depots

In [ ]:
locations_df = (pd.concat([depots_df[["Name","Longitude","Latitude"]], orders_df[["Name","Longitude","Latitude"]]], ignore_index=True)).reset_index()

# Create Cost Matrix

The **cost matrix** models the cost between each pair of locations.  It is used by cuOpt to compute the cost of traveling from any location to any other. The cost matrix needs to be a square matrix of dimension equal to the total number of locations which inlcludes both depots and orders. In this Vehicle Routing Problem, our cost metric is travel time. This is cost we want to minimize. 

To build a a cost matrix of live traffic data, we need to use a third party map data provider. In this workflow, the cost matrix will calculate the travel time in minutes between each two pairs of locations which we build using OSRM. 

In practical applications, you can integrate this to a third-party map data provider like Esri or Google Maps to get live traffic data and run dynamic/real-time re-routing using cuOpt.

In [ ]:
def build_travel_time_matrix(df):
    latitude = df.Latitude.to_numpy()
    longitude = df.Longitude.to_numpy()
    
    locations=""
    n_orders = len(df)
    for i in range(n_orders):
        locations = locations + "{},{};".format(longitude[i], latitude[i])
    r = requests.get("http://router.project-osrm.org/table/v1/car/"+ locations[:-1])
    routes = json.loads(r.content)
    
    # OSRM returns duration in seconds. Here we are converting to minutes
    for i in routes['durations']:
        i[:] = [x / 60 for x in i]
    
    coords_index = { i: (latitude[i], longitude[i]) for i in range(df.shape[0])}
    time_matrix = pd.DataFrame(routes['durations'])
    
    return time_matrix

In [ ]:
def build_cost_matrix(df):

    cost_matrix = distance.cdist(df.values, df.values, "euclidean")
    
    return cost_matrix

In [ ]:
cost_matrix_df = build_cost_matrix(locations_df[['Longitude','Latitude']])

In [ ]:
travel_time_matrix_df = build_travel_time_matrix(locations_df)

### Set Fleet Data

Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

- vehicle_locations is a list of the start and end location of the vehicles. Each vehicle is assigned to a depot from which it departs in the morning and returns to at night. For example, a vehicle that starts and ends in depot 1 which is the location at index 0 would have the vehicle location of [0,0]. 

- capacities is a list of how much goods each vehicle can carry in weight. Here we have two different types of vehicles: trucks and EV vans. A truck can carry up to 20,000 pounds and an EV van can carry up to 8,000 pounds. This is essential when assigning orders to vehicles because one vehicle can only carry so many orders at once. 

- vehicle_time_windows is a list of the integer representation of the operating time of each vehicle. Equivalently, the shift of each vehicle driver. We convert the UTC timestamp to epoch time (integer representation in minutes).

- vehicle_break_time_windows is a list of the integer representation of break time of each vehicle within its operating time. For a driver working an 8 hour shift, this break in the middle of the day represents their lunch break. These time windows are when their lunch break may occur.
  
- vehicle_break_durations is the length of the break. Here, we set the duration to be 30 minutes for all vehicles. 

- vehicle_max_time is a list of the maximum time a vehicle can operate. Even if a driver is available for a long period of time, this constraint enforces a maximum length for a driver's shift. This is also given in minutes. A driver's time window represents total availability which may be longer than a standard shift length. If a driver says they are available to work from 9am to 9pm, we still want to limit their shift to be shorter. A truck driver can drive up to 7 hours, and an EV driver can drive up to 4 hours. 

In [ ]:
depot_names_to_indices_dict = {locations_df["Name"].values.tolist()[i]: i for i in range(n_depots)}
vehicle_locations = vehicles_df[["assigned_depot","assigned_depot"]].replace(depot_names_to_indices_dict).values.tolist()

In [ ]:
capacities = [[int(a) for a in vehicles_df['vehicle_capacity'].tolist()]]

In [ ]:
vehicle_time_windows = pd.concat((vehicles_df['vehicle_start'].apply(get_minutes_from_datetime).to_frame(), vehicles_df['vehicle_end'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()

In [ ]:
vehicle_break_time_windows = [pd.concat((vehicles_df['break_start'].apply(get_minutes_from_datetime).to_frame(), vehicles_df['break_end'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()]

In [ ]:
vehicle_break_durations = [[30] * n_vehicles]

In [ ]:
vehicles_max_time = vehicles_df['max_time'].tolist()

### Set Task Data


Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

- task_locations is the list of stores that have placed an order. This list is simply the index of each location. 

- task_time_windows is the list of integer representation of opening hours for each store. We convert the UTC timestamp to epoch time (integer representation in minutes).

- service_times is the list of the length of time for orders to be dropped off once the vehicle reaches the location. Here, these values are between 15 and 30 minutes.

- demand is the list of weight demand for each order. Here, these values are between 40 and 200 pounds. 

- vehicle_match_list allows us to ensure that some orders are assigned to specific vehicles. In this use case, some of the orders are frozen and can be delivered in trucks and not EV vans. Here we can indicate that the frozen orders are assigned specifically to vehicles that are trucks.  

In [ ]:
task_locations = locations_df.index.tolist()[n_depots:]

In [ ]:
vehicle_locations

In [ ]:
demands = [orders_df['Demand'].values.tolist()]

In [ ]:
service_times = orders_df['ServiceTime'].values.tolist()

In [ ]:
task_time_windows = pd.concat((orders_df['order_start_time'].apply(get_minutes_from_datetime).to_frame(), orders_df['order_end_time'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()

In [ ]:
trucks_ids = vehicles_df['vehicle_type'][vehicles_df['vehicle_type']=="Truck"].index.values.tolist()

In [ ]:
vehicle_match_list = []
for i in orders_df['is_frozen'][orders_df['is_frozen']==1].index.values.tolist():
    vehicle_match_list.append({"order_id": i, "vehicle_ids": trucks_ids})

### Set Solver configuration

Before we send our data to the cuOpt solver, we will add a few several configuration settings.

- the time_limit is the maximum time allotted to find a solution. This depends on the user, who has the flexibility of setting a higher time‑limit for better results. 

- the number of parallel agents (climbers) examining the solution search space.

If the user wants the first solution, then around 2-3 seconds for 2000-4000 climbers are enough. cuOpt solver does not interrupt the initial solution. So if the user specifies a shorter time than it takes for the initial solution, the initial solution is returned when it is computed. Increasing the number of climbers will increase the time it takes to compute the initial solution.
By default, the number of climbers is chosen by considering occupancy of a small GPU and experimented runtime vs number of climbers trade-off (that is, the best result in shortest time). Normally 1024 is a good start.

In [ ]:
# Set the time limit 
# Set number of climbers that will try to search for an optimal routes in parallel
time_limit = 0.1
number_of_climbers = 1024

## Save data in a dictionary

Here, we take all the data we have prepared so far and save it to one dictionary. This includes the cost matrices, task data, fleet data, and solver config. This is all the data that cuOpt needs to solve our LMD problem. 

In [ ]:
cuopt_problem_data = {
    "cost_matrix_data": {
        "cost_matrix": {
            "0": cost_matrix_df.tolist()
        }
    },
    "travel_time_matrix_data": {
        "cost_matrix": {
            "0": travel_time_matrix_df.to_numpy().tolist()
        }
    },
    "task_data": {
        "task_locations": task_locations,
        "demand": demands,
        "task_time_windows": task_time_windows,
        "service_times":service_times,
        "order_vehicle_match": vehicle_match_list,
    },
    "fleet_data": {
        "vehicle_locations": vehicle_locations,
        "capacities": capacities,
        "vehicle_time_windows": vehicle_time_windows,
        "vehicle_break_time_windows": vehicle_break_time_windows,
        "vehicle_break_durations": vehicle_break_durations,
        "vehicle_max_times": vehicles_max_time,
    },
    "solver_config": {
        "time_limit": time_limit,
        "number_of_climbers": number_of_climbers
    }
}

## Create a Service Client Instance

Now that we have prepared all of our data, we can establish a connection to the cuOpt service. 

In the cell below, paste your client ID and Secret that you received via email in order to authenticate the connection. 

Here, we create an instance of the cuOpt Service Client to establish a connection. 


In [ ]:
cuopt_client_id = "<YOUR CLIENT ID>"
cuopt_client_secret = "<YOUR CLIENT SECRET>"

cuopt_service_client = CuOptServiceClient(
    client_id=cuopt_client_id,
    client_secret=cuopt_client_secret,
    )

## Send data to the cuOpt service and get the routes

When using the Python SDK or microservice, we need to send all the data in to cuOpt in multiple steps or API calls. 

When using the managed service, we send all the data in at once.

In [ ]:
# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

location_names = [str(x) for x in locations_df.index.tolist()]

if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

In [ ]:
solver_response

## Visualize the routes


In the drop down menu below, you can select different vehicle ID's to see if they are dispatched. If they are, we print their assigned route on a map.

In this example, vehicle 0 is not dispatched. Choose a vehicle starting from vehicle 1.


Generating a route and map uses third party tools and takes about 30 seconds to run.

In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact

w = widgets.Dropdown(
    options = list(vehicles_df.index.values),
    description='Vehicle ID:',
)

def on_change(value):
    if str(value) in list(solver_resp['vehicle_data'].keys()):
        curr_route_df = pd.DataFrame(solver_resp["vehicle_data"][str(value)]['route'], columns=["stop_index"])
        curr_route_df = pd.merge(curr_route_df, locations_df, how="left", left_on=["stop_index"], right_on=["index"])
        display(get_map_by_vehicle(curr_route_df))        
    else:
        print("This Vehicle is not assigned to any order!!")

interact(on_change, value=w)

# Reoptimization

In this use case, we have provided data for a problem that has a solution. However, in some instances we may have a solution that is infeasible due to demand and slack time constraints. We can relax some of the constraints and reoptimize in order to get a feasible solution. 
Here we are adding a new constraint of penalty and a new solver setting of sift time windows.


Let's start by adding soft time windows. With this, we can relax time window constraints along with a penalty to come up with a solution but at an additional cost. By adding penalties, we can prioritize order/customers by providing higher penalties to such jobs compared to others.

To add soft time windows, we change the solution scope in solver_settings.

In [ ]:
solution_scope = 1

Next, when adding penalty we prioritize orders/customers that are a "loyal member" by providing higher penalties to such tasks compared toother orders/customers that are not "loyalty member".

In [ ]:
loyalty_member = orders_df['loyalty_member'].values.tolist()

penalty = [x * 100 for x in loyalty_member]

In [ ]:
cuopt_problem_data = {
    "cost_matrix_data": {
        "cost_matrix": {
            "0": cost_matrix_df.tolist()
        }
    },
    "travel_time_matrix_data": {
        "cost_matrix": {
            "0": travel_time_matrix_df.to_numpy().tolist()
        }
    },
    "task_data": {
        "task_locations": task_locations,
        "demand": demands,
        "task_time_windows": task_time_windows,
        "service_times":service_times,
        "order_vehicle_match": vehicle_match_list,
        "penalties": penalty,
    },
    "fleet_data": {
        "vehicle_locations": vehicle_locations,
        "capacities": capacities,
        "vehicle_time_windows": vehicle_time_windows,
        "vehicle_break_time_windows": vehicle_break_time_windows,
        "vehicle_break_durations": vehicle_break_durations,
        "vehicle_max_times": vehicles_max_time
    },
    "solver_config": {
        "time_limit": time_limit,
        "number_of_climbers": number_of_climbers,
        "solution_scope": solution_scope,
    }
}

In [ ]:
# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

location_names = [str(x) for x in locations_df.index.tolist()]

if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact

w = widgets.Dropdown(
    options = list(vehicles_df.index.values),
    description='Vehicle ID:',
)

def on_change(value):
    if str(value) in list(solver_resp['vehicle_data'].keys()):
        curr_route_df = pd.DataFrame(solver_resp["vehicle_data"][str(value)]['route'], columns=["stop_index"])
        curr_route_df = pd.merge(curr_route_df, locations_df, how="left", left_on=["stop_index"], right_on=["index"])
        display(get_map_by_vehicle(curr_route_df))        
    else:
        print("This Vehicle is not assigned to any order!!")

interact(on_change, value=w)